In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from models import Models

In [2]:
from aeon.datasets import load_classification

X, y = load_classification("ECG200")

y = np.where(y == '-1', 0, y)
y = y.astype(np.float64)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [3]:
lstm = Models('lstm', X_train, y_train, X_test, y_test)
lstm.train_lstm()
lstm.predict(verbose=True)

lstm Model Accuracy: 0.875


In [4]:
catch22 = Models('catch22', X_train, y_train, X_test, y_test)
catch22.train_catch22()
catch22.predict(verbose=True)

catch22 Model Accuracy: 0.85


In [5]:
rocket = Models('rocket', X_train, y_train, X_test, y_test)
rocket.train_rocket()
rocket.predict(verbose=True)

rocket Model Accuracy: 0.925
